In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting
import matplotlib
matplotlib.style.use('fivethirtyeight')
import os
import seaborn as sns ## plotting histograms

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output

In [ ]:
os.getcwd()

In [2]:
tennis_df = pd.read_csv('Data.csv', encoding = 'latin1') # Read Data

/Users/jayantjuneja/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
tennis_df = pd.read_csv('Data.csv', encoding = 'latin1') # Read Data
tennis_df.WRank = pd.to_numeric(tennis_df.WRank, errors = 'coerce') 
tennis_df.LRank = pd.to_numeric(tennis_df.LRank, errors = 'coerce')
# New Feature: Rank difference betweehn the 2 oponents
tennis_df['Diff'] =  tennis_df.LRank - tennis_df.WRank 
# New Feature: Round the rank difference to 10's and 20's
tennis_df['Round_10'] = 10*round(np.true_divide(tennis_df.Diff,10))
tennis_df['Round_20'] = 20*round(np.true_divide(tennis_df.Diff,20))
# New Feature: Total number of sets in the match
tennis_df['Total Sets'] = tennis_df.Wsets + tennis_df.Lsets

tennis_df.W3 = tennis_df.W3.fillna(0)
tennis_df.W4 = tennis_df.W4.fillna(0)
tennis_df.W5 = tennis_df.W5.fillna(0)
tennis_df.L3 = tennis_df.L3.fillna(0)
tennis_df.L4 = tennis_df.L4.fillna(0)
tennis_df.L5 = tennis_df.L5.fillna(0)

tennis_df['Sets Diff'] = tennis_df.W1+tennis_df.W2+tennis_df.W3+tennis_df.W4+tennis_df.W5 - (tennis_df.L1+tennis_df.L2+tennis_df.L3+tennis_df.L4+tennis_df.L5)
new_df = tennis_df

# 2 New Data Frames: Grand Slam data frame (GS) and non-Grand Slam data frame (non GS)
df_non_GS = new_df[~(new_df.Series == 'Grand Slam')]
df_GS = new_df[new_df.Series == 'Grand Slam']

In [ ]:
tennis_df

In [ ]:
bins = np.arange(10,200,10)
Gs_prob = []

for bi in bins:
    
    pos = bi
    neg = -pos
    
    pos_wins = len(df_GS[df_GS.Round_10 == pos])
    neg_wins = len(df_GS[df_GS.Round_10 == neg])
    Gs_prob.append(np.true_divide(pos_wins,pos_wins + neg_wins))
    

prob = []

for bi in bins:
    
    W = np.true_divide(1,np.mean(df_GS.AvgW[df_GS.Round_10 == bi]))
    L = np.true_divide(1,np.mean(df_GS.AvgL[df_GS.Round_10 == bi]))
    ratio = np.true_divide(1,L + W)
    part_ratio = (ratio - 1)/2 + 1
    prob.append(W/part_ratio)
    
plt.figure(figsize = (10,10))    
#plt.hold(True)
plt.bar(bins,prob, width = 10, color = 'blue')
plt.bar(bins,Gs_prob, width = 8, color = 'red')
plt.xlabel('Rank Difference', fontsize = 15)
plt.ylabel('Winning Probability', fontsize = 15)
plt.xlim([10,200])
plt.ylim([0.5,1])
plt.title('Are Betting Markets Good Predictors in Grand Slam Tournaments?', fontsize = 15)
plt.legend(['Betting Market Odds','Reality'], loc = 2, fontsize = 15)
plt.show()


In [ ]:
prob

**XGBOOSTING**


In [3]:
tennis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46652 entries, 0 to 46651
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ATP         46652 non-null  int64  
 1   Location    46652 non-null  object 
 2   Tournament  46652 non-null  object 
 3   Date        46652 non-null  object 
 4   Series      46652 non-null  object 
 5   Court       46652 non-null  object 
 6   Surface     46652 non-null  object 
 7   Round       46652 non-null  object 
 8   Best of     46652 non-null  int64  
 9   Winner      46652 non-null  object 
 10  Loser       46652 non-null  object 
 11  WRank       46637 non-null  object 
 12  LRank       46581 non-null  object 
 13  W1          46423 non-null  float64
 14  L1          46423 non-null  float64
 15  W2          45962 non-null  float64
 16  L2          45963 non-null  float64
 17  W3          21365 non-null  float64
 18  L3          21365 non-null  float64
 19  W4          4226 non-null